In [165]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [166]:
class DenseLayer(tf.keras.layers.Layer):

    def __init__(self, units=1):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True)

    def call(self, inputs, **kwargs):
        regular = tf.reduce_mean(tf.square(self.w))
        self.add_metric(regular, name='mean square weights')
        self.add_loss(regular)
        return tf.matmul(inputs, self.w) + self.b

In [167]:
layer1 = DenseLayer(10)

In [168]:
y = layer1(tf.constant([[1., 2., 3.]]))

In [169]:
print(y)

tf.Tensor(
[[-0.05643272  0.03796775 -0.13198644  0.04047462  0.09221661 -0.01837657
  -0.03551109  0.01132322  0.53335214  0.19476862]], shape=(1, 10), dtype=float32)


In [170]:
class NeuralNetwork(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_1 = DenseLayer(128)
        self.layer_2 = DenseLayer(10)

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.softmax(x)
        return x

In [171]:
model = NeuralNetwork()

In [172]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
              loss=tf.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [173]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [174]:
x_train = x_train / 255
x_test = x_test / 255

In [175]:
x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

y_train = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [176]:
model.fit(x_train, y_train, batch_size=32, epochs=5)
print(model.evaluate(x_test, y_test_cat))

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3092 - accuracy: 0.9170 - mean square weights: 0.0103
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1516 - accuracy: 0.9620 - mean square weights: 0.0168
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1201 - accuracy: 0.9728 - mean square weights: 0.0197
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1031 - accuracy: 0.9786 - mean square weights: 0.0210
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.1159 - accuracy: 0.9736 - mean square weights: 0.0212
[0.11594245582818985, 0.9735999703407288, 0.02121833525598049, 0.02121833525598049]
